This Notebook provides step by step instructions for ingesting data into a KQL DB

## Prerequisites

1. A workspace with a [Microsoft Fabric-enabled capacity](https://learn.microsoft.com/fabric/enterprise/licenses#capacity). 
2. An [Eventhouse in Microsoft Fabric](https://learn.microsoft.com/fabric/real-time-intelligence/eventhouse). 



## Load data into dataframe


In [ ]:
import pandas as pd

sparkDF = spark.sql("SELECT * FROM Lakehouse.demo_stocks LIMIT 1000")
display(sparkDF)


## Write to Eventhouse

The cluster URI can be found in the Eventhouse [system overview](https://learn.microsoft.com/fabric/real-time-intelligence/manage-monitor-eventhouse#view-system-overview-details-for-an-eventhouse). Enter the name of the database in this Eventhouse to which you will write the data. 

In [ ]:
# replace with your Eventhouse Cluster URI, Database name, and Table name
KUSTO_CLUSTER =  "<your Eventhouse cluster URI>"
KUSTO_DATABASE = "KQL_demos"
KUSTO_TABLE = "Stock"

In [3]:
kustoOptions = {"kustoCluster": KUSTO_CLUSTER, "kustoDatabase" :KUSTO_DATABASE, "kustoTable" : KUSTO_TABLE }

access_token=mssparkutils.credentials.getToken(kustoOptions["kustoCluster"])

StatementMeta(, d01b6a8d-d956-439d-9739-1e11c389055f, 5, Finished, Available, Finished)

In [7]:
# Write data to a table in Eventhouse
sparkDF.write. \
format("com.microsoft.kusto.spark.synapse.datasource"). \
option("kustoCluster",kustoOptions["kustoCluster"]). \
option("kustoDatabase",kustoOptions["kustoDatabase"]). \
option("kustoTable", kustoOptions["kustoTable"]). \
option("accessToken", access_token). \
option("tableCreateOptions", "CreateIfNotExist").\
mode("Append"). \
save()

StatementMeta(, d01b6a8d-d956-439d-9739-1e11c389055f, 9, Finished, Available, Finished)

In [8]:
# read 100 rows from Eventhouse
kustoQuery = "Stock | take 100" 
accessToken = mssparkutils.credentials.getToken(KUSTO_CLUSTER)
kustoDf  = spark.read\
    .format("com.microsoft.kusto.spark.synapse.datasource")\
    .option("accessToken", accessToken)\
    .option("kustoCluster", KUSTO_CLUSTER)\
    .option("kustoDatabase", KUSTO_DATABASE)\
    .option("kustoQuery", kustoQuery).load()

# Example that uses the result data frame.
kustoDf.show()

StatementMeta(, d01b6a8d-d956-439d-9739-1e11c389055f, 10, Finished, Available, Finished)

+-------------------+-------------------+------------------+----+-------------------+-------------------+
|               Date|               AAPL|              AMZN|GOOG|               MSFT|                SPY|
+-------------------+-------------------+------------------+----+-------------------+-------------------+
|2017-02-22 00:00:00|  0.146808305449985| 0.955419760494327| 0.0| -0.433367554019207|  0.212296191801005|
|2017-10-10 00:00:00|  0.160451834874065| 0.345333726944671| 0.0|   0.47387206077003|-0.0117813214160214|
|2018-02-15 00:00:00|      4.14008849237|               0.0| 0.0|   3.05049924649926|                0.0|
|2019-02-08 00:00:00|  -1.97795158007245|              -2.4| 0.0|  -0.76053145088901|                0.0|
|2020-10-09 00:00:00| -0.834409652217742|-0.464807341410868| 0.0|  0.342027095858265|  0.790430627631345|
|2020-10-27 00:00:00|   1.29812797106224| 0.819070985821367| 0.0|  -1.05682006131486| -0.692717704779877|
|2020-11-19 00:00:00| -0.859964815118564|-0.91